In [15]:
import numpy as np
import pandas as pd
import datetime

import matplotlib
import matplotlib.pyplot as plt

In [16]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn import metrics
from sklearn.metrics import fbeta_score, make_scorer

In [17]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [18]:
def Mape(ground_truth, predictions):
    diff = (ground_truth - predictions) / ground_truth
    diff = np.abs(diff)
    mape = diff.sum() / len(ground_truth)
    print('mape=%f'%mape)
    return mape

Mapeloss = make_scorer(Mape, greater_is_better=False)

In [19]:
validation_path = '../../data/online/'

In [20]:
link_df = pd.read_csv('../../data/original/links (table 3).csv', index_col=0)

In [21]:
morning_gbdt_regressor_dict = {}
for link_id in link_df.index.values:

    X_train_df = pd.read_csv(validation_path + 'feature/feature2/morning/X_train_link' + str(link_id) + '.csv', index_col = 0)
    y_train_df = pd.read_csv(validation_path + 'feature/feature2/morning/y_train_link' + str(link_id) + '.csv', index_col = 0)
    morning_gbdt_regressor_dict[link_id] = []
    for t in range(6):
        X_train, X_validation, y_train, y_validation = train_test_split(X_train_df.values, y_train_df[str(t)].values, random_state=7, test_size=0.5)
        gbdt_regressor = GradientBoostingRegressor(learning_rate=0.01, n_estimators=8,min_samples_split=3, min_samples_leaf=2)
        gbdt_regressor.fit(X_train, y_train)
        predict = gbdt_regressor.predict(X_validation)
        morning_gbdt_regressor_dict[link_id].append(gbdt_regressor)
        

In [22]:
afternoon_gbdt_regressor_dict = {}
for link_id in link_df.index.values:

    X_train_df = pd.read_csv(validation_path +  'feature/feature2/afternoon/X_train_link' + str(link_id) + '.csv', index_col = 0)
    y_train_df = pd.read_csv(validation_path +  'feature/feature2/afternoon/y_train_link' + str(link_id) + '.csv', index_col = 0)
    afternoon_gbdt_regressor_dict[link_id] = []
    for t in range(6):
        X_train, X_validation, y_train, y_validation = train_test_split(X_train_df.values, y_train_df[str(t)].values, random_state=7, test_size=0.5)
        gbdt_regressor = GradientBoostingRegressor(learning_rate=0.01, n_estimators=8,min_samples_split=3, min_samples_leaf=2)
        gbdt_regressor.fit(X_train, y_train)
        predict = gbdt_regressor.predict(X_validation)
        afternoon_gbdt_regressor_dict[link_id].append(gbdt_regressor)
        

# Test集

In [23]:
morning_predict_link_arrive_time_dict = {}
for link_id in link_df.index.values:

    X_test_df = pd.read_csv(validation_path + 'feature/feature2/morning/X_test_link' + str(link_id) + '.csv', index_col = 0)
    morning_predict_link_arrive_time_dict[link_id] = []
    for d in range(len(X_test_df)):
        predicts = []
        for t in range(6):
            gbdt_regressor = morning_gbdt_regressor_dict[link_id][t]
            p = gbdt_regressor.predict(X_test_df.loc[d].values.reshape(1, -1))
            predicts.extend(p)
        morning_predict_link_arrive_time_dict[link_id].append(predicts)

In [24]:
afternoon_predict_link_arrive_time_dict = {}
for link_id in link_df.index.values:

    X_test_df = pd.read_csv(validation_path + 'feature/feature2/afternoon/X_test_link' + str(link_id) + '.csv', index_col = 0)
    afternoon_predict_link_arrive_time_dict[link_id] = []
    for d in range(len(X_test_df)):
        predicts = []
        for t in range(6):
            gbdt_regressor = afternoon_gbdt_regressor_dict[link_id][t]
            p = gbdt_regressor.predict(X_test_df.loc[d].values.reshape(1, -1))
            predicts.extend(p)
            
        afternoon_predict_link_arrive_time_dict[link_id].append(predicts)

In [25]:
for link_id in link_df.index.values:
    predict_df = pd.DataFrame(afternoon_predict_link_arrive_time_dict[link_id])
    predict_df.to_csv(validation_path + 'prediction/feature2/afternoon/' + str(link_id) + '.csv')
    
    predict_df = pd.DataFrame(morning_predict_link_arrive_time_dict[link_id])
    predict_df.to_csv(validation_path + 'prediction/feature2/morning/' + str(link_id) + '.csv')

In [11]:
routes = pd.read_csv('../data/original/routes (table 4).csv')

In [12]:
submissions = []
for i in routes.index:
    link_seq = routes.loc[i]['link_seq']
    links = link_seq.split(',')
    
    #morning
    start_time = pd.to_datetime('2016-10-18 08:00:00')
    for d in range(7):
        current_time = start_time
        for t in range(6):
            time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                time = time + morning_predict_link_arrive_time_dict[int(link_id)][d][t]
            record.append(time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)
        
    #afternoon
    start_time = pd.to_datetime('2016-10-18 17:00:00')
    for d in range(7):
        current_time = start_time
        for t in range(6):
            time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                time = time + afternoon_predict_link_arrive_time_dict[int(link_id)][d][t]
            record.append(time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)

In [13]:
predict_df = pd.DataFrame(submissions).rename(columns={0:'intersection_id', 1:'tollgate_id', 2:'time_window', 3:'avg_travel_time'})

In [14]:
predict_df.to_csv('../data/prediction/prediction_m2.csv', index=False)